In [ ]:
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import mobilenet_v2, resnet34
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import random
import joblib

In [2]:
# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# task0_resnet34_cifar10.npy - test output

# Load data
DATA_PATH = './pickle/cifar10/resnet34/test.p'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
print(type(dataset), len(dataset))

# load target model
MODEL_PATH = './models/resnet34_cifar10.pth'
target_model = resnet34(num_classes=10).to(device)
state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

# preparing the test data
images = [item[0] for item in dataset]
labels = [item[1] for item in dataset]
images_tensor = torch.stack(images)
labels_tensor = torch.tensor(labels)
dataset_tensor = TensorDataset(images_tensor, labels_tensor)
test_loader = DataLoader(dataset_tensor, batch_size=64, shuffle=False, num_workers=2)

#  Generate test data outputs using target model
target_model.eval()
probabilities = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())
X_test = np.array(probabilities)

# loading attack model
attack_model = joblib.load('./attack_models/cifar10_resnet/attack_model_74.pkl')

# evaluate the attack model on the test data
predictions = attack_model.predict(X_test)

# save the predictions
predictions = np.array(predictions)
predictions = predictions.astype(int)
np.save('./results/task0_resnet34_cifar10.npy', predictions)
print('Predictions saved to task0_resnet34_cifar10.npy')



cuda:0
<class 'list'> 20000
Predictions saved to task0_resnet34_cifar10.npy


c:\Users\fahim\miniconda3\envs\torch\lib\site-packages\xgboost\core.py:160: UserWarning: [18:31:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [8]:
# task1_mobilenetv2_cifar10.npy - test output

# Load data
DATA_PATH = './pickle/cifar10/mobilenetv2/test.p'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
print(type(dataset), len(dataset))

# load target model
MODEL_PATH = './models/mobilenetv2_cifar10.pth'
target_model = mobilenet_v2(num_classes=10).to(device)
state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

# preparing the test data
images = [item[0] for item in dataset]
labels = [item[1] for item in dataset]
images_tensor = torch.stack(images)
labels_tensor = torch.tensor(labels)
dataset_tensor = TensorDataset(images_tensor, labels_tensor)
test_loader = DataLoader(dataset_tensor, batch_size=64, shuffle=False, num_workers=2)

#  Generate test data outputs using target model
target_model.eval()
probabilities = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())
X_test = np.array(probabilities)

# loading attack model
attack_model = joblib.load('./attack_models/mobilenetv2_cifar10/attack_model_69.pkl')

# evaluate the attack model on the test data
predictions = attack_model.predict(X_test)

# save the predictions
predictions = np.array(predictions)
predictions = predictions.astype(int)
np.save('./results/task1_mobilenetv2_cifar10.npy', predictions)
print('Predictions saved to task1_mobilenetv2_cifar10.npy')



cuda:0
<class 'list'> 20000
Predictions saved to task1_mobilenetv2_cifar10.npy


In [26]:
# task2_resnet34_tinyimagenet.npy - test output

# Load data
DATA_PATH = './pickle/tinyimagenet/resnet34/test.p'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
print(type(dataset), len(dataset))

# load target model
MODEL_PATH = './models/resnet34_tinyimagenet.pth'
target_model = resnet34(num_classes=200).to(device)
state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

# preparing the test data
images = [item[0] for item in dataset]
labels = [item[1] for item in dataset]
images_tensor = torch.stack(images)
labels_tensor = torch.tensor(labels)
dataset_tensor = TensorDataset(images_tensor, labels_tensor)
test_loader = DataLoader(dataset_tensor, batch_size=64, shuffle=False, num_workers=2)

#  Generate test data outputs using target model
target_model.eval()
probabilities = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())
X_test = np.array(probabilities)

# loading attack model
class AttackModel(nn.Module):
    def __init__(self):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(200, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.dropout(nn.functional.relu(self.fc2(x)))
        x = self.dropout(nn.functional.relu(self.fc3(x)))
        x = nn.functional.sigmoid(self.fc4(x))
        return x
    
attack_model_nn = AttackModel().to(device)
MODEL_PATH = './attack_models/tinyimagenet_resnet/attack_model_99.pt'
attack_model_nn.load_state_dict(torch.load(MODEL_PATH))

# evaluate the attack model on the test data
attack_model_nn.eval()
predictions = []
with torch.no_grad():
    outputs = attack_model_nn(torch.tensor(X_test, dtype=torch.float32).to(device))
    y_pred = outputs.round().view(-1).cpu().numpy()
    predictions.extend(y_pred)


# save the predictions
predictions = np.array(predictions)
predictions = predictions.astype(int)
np.save('./results/task2_resnet34_tinyimagenet.npy', predictions)
print('Predictions saved to task2_resnet34_tinyimagenet.npy')



cuda:0
<class 'list'> 20000
Predictions saved to task2_resnet34_tinyimagenet.npy


In [27]:
# task3_mobilenetv2_tinyimagenet.npy - test output

# Load data
DATA_PATH = './pickle/tinyimagenet/mobilenetv2/test.p'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
print(type(dataset), len(dataset))

# load target model
MODEL_PATH = './models/mobilenetv2_tinyimagenet.pth'
target_model = mobilenet_v2(num_classes=200).to(device)
state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

# preparing the test data
images = [item[0] for item in dataset]
labels = [item[1] for item in dataset]
images_tensor = torch.stack(images)
labels_tensor = torch.tensor(labels)
dataset_tensor = TensorDataset(images_tensor, labels_tensor)
test_loader = DataLoader(dataset_tensor, batch_size=64, shuffle=False, num_workers=2)

#  Generate test data outputs using target model
target_model.eval()
probabilities = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())
X_test = np.array(probabilities)

# loading attack model
class AttackModel(nn.Module):
    def __init__(self):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(200, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc5 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.dropout(nn.functional.relu(self.bn1(self.fc1(x))))
        x = self.dropout(nn.functional.relu(self.bn2(self.fc2(x))))
        x = self.dropout(nn.functional.relu(self.bn3(self.fc3(x))))
        x = self.dropout(nn.functional.relu(self.bn4(self.fc4(x))))
        x = nn.functional.sigmoid(self.fc5(x))
        return x
    
attack_model_nn = AttackModel().to(device)
MODEL_PATH = './attack_models/mobilenetv2_tinyimagenet/attack_model_95.pt'
attack_model_nn.load_state_dict(torch.load(MODEL_PATH))

# evaluate the attack model on the test data
attack_model_nn.eval()
predictions = []
with torch.no_grad():
    outputs = attack_model_nn(torch.tensor(X_test, dtype=torch.float32).to(device))
    y_pred = outputs.round().view(-1).cpu().numpy()
    predictions.extend(y_pred)


# save the predictions
predictions = np.array(predictions)
predictions = predictions.astype(int)
np.save('./results/task3_mobilenetv2_tinyimagenet.npy', predictions)
print('Predictions saved to task3_mobilenetv2_tinyimagenet.npy')



cuda:0
<class 'list'> 20000
Predictions saved to task3_mobilenetv2_tinyimagenet.npy


In [17]:
# check the outputs in results folder
predictions_0 = np.load('./results/task0_resnet34_cifar10.npy')
predictions_0_sub1 = np.load('./results/task0_resnet34_cifar10_sub1.npy')
predictions_1 = np.load('./results/task1_mobilenetv2_cifar10.npy')
predictions_1_sub1 = np.load('./results/task1_mobilenetv2_cifar10_sub1.npy')
predictions_2 = np.load('./results/task2_resnet34_tinyimagenet.npy')
predictions_3 = np.load('./results/task3_mobilenetv2_tinyimagenet.npy')

print(predictions_0.shape, predictions_1.shape, predictions_2.shape, predictions_3.shape)
print(predictions_0[:10])
print(predictions_0_sub1[:10])
print(predictions_1[:10])
print(predictions_1_sub1[:10])
print(predictions_2[:10])
print(predictions_3[:10])

(20000,) (20000,) (20000,) (20000,)
[0 0 0 1 1 0 1 0 0 0]
[1 0 0 1 1 1 1 1 0 0]
[0 1 1 0 1 1 1 1 1 1]
[1 1 1 0 1 1 1 1 1 1]
[0 0 0 0 1 1 1 1 1 1]
[0 0 1 1 1 0 1 1 0 1]
